In [0]:
from rake_nltk import Rake
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Machine Learning Data/final_data.csv')
df.head()


,Unnamed: 0,userId,movieId,imdbId,title,rating,genres,Director_x,Plot_x,Director_y,Plot_y,Cast
0,0,3,1,114709,Toy Story (1995),4.0,Adventure|Animation|Children|Comedy|Fantasy,John Lasseter,A cowboy doll is profoundly threatened and jea...,John Lasseter,A cowboy doll is profoundly threatened and jea...,Tom Hanks
1,1,6,1,114709,Toy Story (1995),5.0,Adventure|Animation|Children|Comedy|Fantasy,John Lasseter,A cowboy doll is profoundly threatened and jea...,John Lasseter,A cowboy doll is profoundly threatened and jea...,Tom Hanks
2,2,8,1,114709,Toy Story (1995),4.0,Adventure|Animation|Children|Comedy|Fantasy,John Lasseter,A cowboy doll is profoundly threatened and jea...,John Lasseter,A cowboy doll is profoundly threatened and jea...,Tom Hanks
3,3,10,1,114709,Toy Story (1995),4.0,Adventure|Animation|Children|Comedy|Fantasy,John Lasseter,A cowboy doll is profoundly threatened and jea...,John Lasseter,A cowboy doll is profoundly threatened and jea...,Tom Hanks
4,4,11,1,114709,Toy Story (1995),4.5,Adventure|Animation|Children|Comedy|Fantasy,John Lasseter,A cowboy doll is profoundly threatened and jea...,John Lasseter,A cowboy doll is profoundly threatened and jea...,Tom Hanks


In [0]:
df1 = df.drop_duplicates('imdbId')
df1.head()

,Unnamed: 0,userId,movieId,imdbId,title,rating,genres,Director_x,Plot_x,Director_y,Plot_y,Cast
0,0,3,1,114709,Toy Story (1995),4.0,Adventure|Animation|Children|Comedy|Fantasy,John Lasseter,A cowboy doll is profoundly threatened and jea...,John Lasseter,A cowboy doll is profoundly threatened and jea...,Tom Hanks
49695,49695,1,2,113497,Jumanji (1995),3.5,Adventure|Children|Fantasy,Joe Johnston,When two kids find and play a magical board ga...,Joe Johnston,When two kids find and play a magical board ga...,Robin Williams
71938,71938,2,3,113228,Grumpier Old Men (1995),4.0,Comedy|Romance,Howard Deutch,John and Max resolve to save their beloved bai...,Howard Deutch,John and Max resolve to save their beloved bai...,Walter Matthau
84673,84673,41,4,114885,Waiting to Exhale (1995),2.0,Comedy|Drama|Romance,Forest Whitaker,"Based on Terry McMillan's novel, this film fol...",Forest Whitaker,"Based on Terry McMillan's novel, this film fol...",Whitney Houston
87429,87429,12,5,113041,Father of the Bride Part II (1995),2.0,Comedy,Charles Shyer,George Banks must deal not only with the pregn...,Charles Shyer,George Banks must deal not only with the pregn...,Steve Martin


In [0]:
df1.shape

(26744, 12)

In [0]:
df1 = df1[['title','Director_x','Cast','genres','Plot_x']]
df1 = df1.reset_index(drop=True)
df1['genres'] = df1['genres'].map(lambda x: x.lower().split('|')[:2])
df1.head()

,title,Director_x,Cast,genres,Plot_x
0,Toy Story (1995),John Lasseter,Tom Hanks,"[adventure, animation]",A cowboy doll is profoundly threatened and jea...
1,Jumanji (1995),Joe Johnston,Robin Williams,"[adventure, children]",When two kids find and play a magical board ga...
2,Grumpier Old Men (1995),Howard Deutch,Walter Matthau,"[comedy, romance]",John and Max resolve to save their beloved bai...
3,Waiting to Exhale (1995),Forest Whitaker,Whitney Houston,"[comedy, drama]","Based on Terry McMillan's novel, this film fol..."
4,Father of the Bride Part II (1995),Charles Shyer,Steve Martin,[comedy],George Banks must deal not only with the pregn...


In [0]:
df1['Plot_x']=df1['Plot_x'].str.split('::').str[0]

df1['Plot_x'] = df1['Plot_x'].map(lambda x: x.lower())

df1.head()

,title,Director_x,Cast,genres,Plot_x
0,Toy Story (1995),John Lasseter,Tom Hanks,"[adventure, animation]",a cowboy doll is profoundly threatened and jea...
1,Jumanji (1995),Joe Johnston,Robin Williams,"[adventure, children]",when two kids find and play a magical board ga...
2,Grumpier Old Men (1995),Howard Deutch,Walter Matthau,"[comedy, romance]",john and max resolve to save their beloved bai...
3,Waiting to Exhale (1995),Forest Whitaker,Whitney Houston,"[comedy, drama]","based on terry mcmillan's novel, this film fol..."
4,Father of the Bride Part II (1995),Charles Shyer,Steve Martin,[comedy],george banks must deal not only with the pregn...


In [0]:
df1['Director_x'] = df1['Director_x'].map(lambda x: x.split(' '))
df1['Cast'] = df1['Cast'].map(lambda x: x.split(' '))

for index, row in df1.iterrows():
    row['Director_x'] = ''.join(row['Director_x']).lower()
    row['Cast'] = ''.join(row['Cast']).lower()
    
df1.head()

,title,Director_x,Cast,genres,Plot_x
0,Toy Story (1995),johnlasseter,tomhanks,"[adventure, animation]",a cowboy doll is profoundly threatened and jea...
1,Jumanji (1995),joejohnston,robinwilliams,"[adventure, children]",when two kids find and play a magical board ga...
2,Grumpier Old Men (1995),howarddeutch,waltermatthau,"[comedy, romance]",john and max resolve to save their beloved bai...
3,Waiting to Exhale (1995),forestwhitaker,whitneyhouston,"[comedy, drama]","based on terry mcmillan's novel, this film fol..."
4,Father of the Bride Part II (1995),charlesshyer,stevemartin,[comedy],george banks must deal not only with the pregn...


In [0]:
df1['Key_words'] = ""

for index, row in df1.iterrows():
    plot = row['Plot_x']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary with key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df1.drop(columns = ['Plot_x'], inplace = True)

In [0]:
df1.set_index('title', inplace = True)
df1.head()

,Director_x,Cast,genres,Key_words
title,,,,
Toy Story (1995),johnlasseter,tomhanks,"[adventure, animation]","[room, profoundly, threatened, new, spaceman, ..."
Jumanji (1995),joejohnston,robinwilliams,"[adventure, children]","[magical, board, game, decades, --, two, kids,..."
Grumpier Old Men (1995),howarddeutch,waltermatthau,"[comedy, romance]","[italian, restaurant, beloved, bait, shop, att..."
Waiting to Exhale (1995),forestwhitaker,whitneyhouston,"[comedy, drama]","[film, follows, four, male, gender, terry, mcm..."
Father of the Bride Part II (1995),charlesshyer,stevemartin,[comedy],"[wife, pregnancy, unexpected, daughter, george..."


In [0]:
df2 = df1.copy()

df2.head()

,Director_x,Cast,genres,Key_words
title,,,,
Toy Story (1995),johnlasseter,tomhanks,"[adventure, animation]","[room, profoundly, threatened, new, spaceman, ..."
Jumanji (1995),joejohnston,robinwilliams,"[adventure, children]","[magical, board, game, decades, --, two, kids,..."
Grumpier Old Men (1995),howarddeutch,waltermatthau,"[comedy, romance]","[italian, restaurant, beloved, bait, shop, att..."
Waiting to Exhale (1995),forestwhitaker,whitneyhouston,"[comedy, drama]","[film, follows, four, male, gender, terry, mcm..."
Father of the Bride Part II (1995),charlesshyer,stevemartin,[comedy],"[wife, pregnancy, unexpected, daughter, george..."


In [0]:
df2['bag_of_words'] = ''
columns = df2.columns
for index, row in df2.iterrows():
    words = ''
    for col in columns:
        if col == 'Director':
            words = words + row[col]+ ' '
        elif col == 'Cast':
            words = words + row[col]+ ' '
        elif col == 'genres':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + ' '.join(row[col])+ ' '
        
        
            
    row['bag_of_words'] = words
    
df2.drop(columns = [col for col in df2.columns if col!= 'bag_of_words'], inplace = True)

df2.head()



,bag_of_words
title,
Toy Story (1995),j o h n l a s s e t e r tomhanks adventure ani...
Jumanji (1995),j o e j o h n s t o n robinwilliams adventure ...
Grumpier Old Men (1995),h o w a r d d e u t c h waltermatthau comedy r...
Waiting to Exhale (1995),f o r e s t w h i t a k e r whitneyhouston com...
Father of the Bride Part II (1995),c h a r l e s s h y e r stevemartin comedy wif...


In [0]:
df2.to_csv('/content/drive/My Drive/Colab Notebooks/Machine Learning Data/bag_of_words_final.csv')
    

In [0]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df2['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df2.index)
indices[:5]
        

0                      Toy Story (1995)
1                        Jumanji (1995)
2               Grumpier Old Men (1995)
3              Waiting to Exhale (1995)
4    Father of the Bride Part II (1995)
Name: title, dtype: object

In [0]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.05892557, 0.05892557, ..., 0.06681531, 0.        ,
        0.06933752],
       [0.05892557, 1.        , 0.        , ..., 0.06299408, 0.        ,
        0.06537205],
       [0.05892557, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.06681531, 0.06299408, 0.        , ..., 1.        , 0.        ,
        0.07412493],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.06933752, 0.06537205, 0.        , ..., 0.07412493, 0.        ,
        1.        ]])

In [0]:
#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df2.index)[i])
        
    return recommended_movies

In [0]:
recommendations('Chinatown (1974)')

['Big Sleep, The (1946)',
 'Laura (1944)',
 'Murder, My Sweet (1944)',
 'Touch of Evil (1958)',
 'Lady in the Lake (1947)',
 'Long Goodbye, The (1973)',
 'Slam Dance (1987)',
 'Detour (1945)',
 'Return of Frank Cannon, The (1980)',
 'Presumed Innocent (1990)']

In [0]:
recommendations('Iron Man (2008)')

['Vikings, The (1958)',
 'Operación Cannabis (2009)',
 'Iron Man 2 (2010)',
 'Spy Kids: All the Time in the World in 4D (2011)',
 'Descendant of the Sun (Ri jie) (1983)',
 'Iron Man 3 (2013)',
 'Rambo: First Blood Part II (1985)',
 'Avengers, The (2012)',
 'Zatoichi and the Fugitives (Zatôichi hatashi-jô) (Zatôichi 18) (1968)',
 'BloodRayne: The Third Reich (2011)']